In [ ]:
import os
import pandas as pd

In [ ]:
path = os.getcwd()
files = os.listdir(path)
files

In [ ]:
files_csv = [f for f in files if f[-4:] == '.csv']
files_csv

In [ ]:
df = pd.DataFrame()

In [ ]:
for f in files_csv:
    data = pd.read_csv(f, 'rawdata')
    df = df.append(data)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df1 = pd.read_csv("allData.csv",usecols=['Description', 'Product', 'Reason For Calling'])

In [ ]:
df2 = pd.read_csv("Closed Duplicates Cases from Accounts - August 2018 - Sheet1.csv",usecols=['Description','Problem Category', 'Reason for calling'])

In [ ]:
df1.columns=['Description','Problem Category', 'Reason for calling']

In [ ]:
df2.columns

In [ ]:
df = pd.DataFrame()

In [ ]:
df = df1.append(df2)

In [ ]:
df = df.dropna(axis = 0, subset=['Description'])

In [ ]:
def trimAllColumns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    df['Description'] = df.Description.replace("\n", " ")
    trimStrings = lambda x: x.strip() if type(x) is str else x
    return df.applymap(trimStrings)


# simple example of trimming whitespace from data elements

df = trimAllColumns(df)

In [ ]:
#!/usr/bin/env python
#coding:utf-8
# Author: Alejandro Nolla - z0mbiehunt3r
# Purpose: Example for detecting language using a stopwords based approach
# Created: 15/05/13

import sys

try:
    from nltk import wordpunct_tokenize
    from nltk.corpus import stopwords
except ImportError:
    print ('[!] You need to install nltk (http://nltk.org/index.html)')



#----------------------------------------------------------------------
def _calculate_languages_ratios(text):
    """
    Calculate probability of given text to be written in several languages and
    return a dictionary that looks like {'french': 2, 'spanish': 4, 'english': 0}
    
    @param text: Text whose language want to be detected
    @type text: str
    
    @return: Dictionary with languages and unique stopwords seen in analyzed text
    @rtype: dict
    """

    languages_ratios = {}

    '''
    nltk.wordpunct_tokenize() splits all punctuations into separate tokens
    
    >>> wordpunct_tokenize("That's thirty minutes away. I'll be there in ten.")
    ['That', "'", 's', 'thirty', 'minutes', 'away', '.', 'I', "'", 'll', 'be', 'there', 'in', 'ten', '.']
    '''

    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens]

    # Compute per language included in nltk number of unique stopwords appearing in analyzed text
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        words_set = set(words)
        common_elements = words_set.intersection(stopwords_set)

        languages_ratios[language] = len(common_elements) # language "score"

    return languages_ratios


#----------------------------------------------------------------------
def detect_language(text):
    """
    Calculate probability of given text to be written in several languages and
    return the highest scored.
    
    It uses a stopwords based approach, counting how many unique stopwords
    are seen in analyzed text.
    
    @param text: Text whose language want to be detected
    @type text: str
    
    @return: Most scored language guessed
    @rtype: str
    """

    ratios = _calculate_languages_ratios(text)

    most_rated_language = max(ratios, key=ratios.get)

    return most_rated_language


In [ ]:
import string
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re

def preprocess(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [w for w in tokens if not w in stopwords.words('english')]
    return " ".join(filtered_words)




for index, row in df['Description'].iteritems():
        #print(row)
        if(row!=" "):
            lang = detect_language(row)
            df.loc[index, 'Language'] = lang
            #print (lang)
            #cleanData = preprocess(row)
            #df.loc[index, 'Clean_Desc'] = cleanData

In [ ]:
df_eng = df[df.Language=="english"]

In [ ]:
import re
from email_reply_parser import EmailReplyParser
for index, row in df_eng['Description'].iteritems():
        if(row!=" "):
            abc = EmailReplyParser.parse_reply(row)
            data = re.sub(r'<.*?>', '', abc)
            df_eng.loc[index, 'primaryMail'] = data
            print(index)
            print("**********************************")
            print(data)
            

In [ ]:
df_eng.isna().sum()

In [ ]:
df_eng.primaryMail.fillna(0,inplace=True)

In [ ]:
from bs4 import BeautifulSoup

def checkNa(row):
    #print(row['primaryMail'])
    #print(row['primaryMail'])

    if(row['primaryMail']==0):
        print(row['Description'])
        soup = BeautifulSoup(row['Description'])
        print(soup.get_text())
        abc = EmailReplyParser.parse_reply(soup.get_text())
        data = re.sub(r'<.*?>', '', abc)
        row['primaryMail'] = data
        return row


In [ ]:
df_eng.apply(checkNa,axis=1)

In [ ]:
def detectlang(row):
    try:
        lang = detect(row['primaryMail'])
        print(lang)
        row['Language'] = lang
    except:
        row['Language'] = "error"
    return row
    

In [ ]:
df_eng = df_eng.apply(detectlang,axis=1)

In [ ]:
df_eng.Language.value_counts()

In [ ]:
df_eng = df_eng[df_eng.Language=="en"]

In [ ]:
df_eng.Language.value_counts()

In [ ]:
df_eng['primaryMail'] = df_eng.primaryMail.replace("?", " ")

In [ ]:
import numpy as np
for index, row in df_eng['primaryMail'].iteritems():
    #print(row)
    print(index)
    if(row!=" "):
        mailWithOnlyAlphabet = re.sub(r'[^a-zA-Z]', ' ',row)
        mail = re.sub("\s\s+", " ", mailWithOnlyAlphabet)
        df_eng.loc[index, 'mail'] = mail
        df_eng.loc[index, 'length'] = len(mail)
        print("******************")

In [ ]:
df_eng = trimAllColumns(df_eng)

In [ ]:
df_eng.length = df_eng.length.astype(int) 

In [ ]:
df_eng = df_eng[df_eng.length>8]

In [ ]:
df_eng.mail = df_eng.mail.str.lower()

In [ ]:
replace_set = {'forwarded','message','lz','logitech','dear', 'my', 'date', 'i', 'recently', 'hi', 'hello', 'product', 'serial', 'number', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', 'purchased', 
               'purchase', 'support', 'http', 'com', 'logitech', 'www', 'https', 'logi', 'customercare',
               'contact', 'terms', 'blvd', 'gateway', 'newark', 'usa', 'logo' ,'care', 'ca', 'footer', 'use', 
               'customer', 'owned', 'us', 'survey', 'americas', 'copyright', 'headquarters', 'owners', 'respective',
               'the','rights', 'trademarks', 'reserved', 'property','dear','regards','thanks', 'mail', 'email','product','serial','number',
              'lz','g','x','k','date','like','get','one','set','thank','also','two',
              'see','able','n','could',
              'since','last','know','still','got','pm','p','n','s'
              'operating','system','platform','ce','s','hs'}

df_eng['mail_text'] = df_eng['mail'].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in replace_set))

df_eng['mail_text'] = df_eng['mail_text'].str.replace('\d+', '')

In [ ]:
def findLength(row):
    row['length']=len(row['mail_text'])
    return row

In [ ]:
df_eng = df_eng.apply(findLength,axis=1)

In [ ]:
df_eng = df_eng[df_eng.length>20]

In [ ]:
def preprocess(row):
    sentence = row['mail_text']
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [w for w in tokens if not w in stopwords.words('english')]
    row['clean'] = tokens
    return row


In [ ]:
df_eng = df_eng.apply(preprocess,axis=1)

In [ ]:
def findLength(row):
    row['length']=len(row['clean'])
    return row

In [ ]:
df_eng = df_eng.apply(findLength,axis=1)

In [ ]:
df_eng = df_eng.reset_index(drop=True)

In [ ]:
df_eng.to_csv("full_preprocessedData_4-10-2018.csv",index=False)